# Figures for the paper that do not use data from evolutionary runs

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import pickle
import itertools

In [ ]:
import importlib
import evotsc
import evotsc_run
import evotsc_plot
importlib.reload(evotsc)
importlib.reload(evotsc_run)
importlib.reload(evotsc_plot)

In [ ]:
## Constants
# Population
nb_indivs = 100
nb_genes = 60

# Genome
intergene = 1000
gene_length = 1000
interaction_dist = 5000
interaction_coef = 0.03
sigma_basal = -0.066
sigma_opt = -0.042
epsilon = 0.005
m = 2.5
default_basal_expression = (1 + np.exp(- m)) / 2 # Average of the maximum and minimum expression levels in the model

# Fitness
selection_coef = 50

# Selection
selection_method = "fit-prop" # Choices: "fit-prop", "rank", "exp-rank"

# Environment
sigma_A = 0.01
sigma_B = -0.01

# Mutations
inversion_poisson_lam = 2.0
intergene_poisson_lam = 0.0 #2.0
intergene_mutation_var = 0.0 #1e1
basal_sc_mutation_prob = 0.0 #1e-1
basal_sc_mutation_var = 0.0 #1e-4

In [ ]:
gene_types = ['AB', 'A', 'B'] # Name of each gene type
gene_type_color = ['tab:blue', 'tab:red', 'tab:green'] #AB, A, B
dpi = 300

In [ ]:
rng = np.random.default_rng(seed=123456)

# Helper functions

In [ ]:
def plot_expr(indiv, sigma_env, plot_title=None, plot_name=None):
    
    if not indiv.already_evaluated:
        indiv.evaluate(sigma_env, sigma_env)
    
    # Plot only environment A
    temporal_expr = indiv.run_system(sigma_env)

    nb_genes, nb_steps = temporal_expr.shape

    colors = mpl.cm.get_cmap('viridis', indiv.nb_genes)(range(indiv.nb_genes))

    plt.figure(figsize=(8, 5), dpi=dpi)

    plt.ylim(-0.05, 1.05)
    
    # Add 1/2 expression level
    for gene in range(indiv.nb_genes):
        linestyle = 'solid' if indiv.genes[gene].orientation == 0 else 'dashed'
        plt.plot(temporal_expr[:, indiv.genes[gene].id],
                 linestyle=linestyle,
                 linewidth=2,
                 zorder=3,
                 color=colors[indiv.genes[gene].id]) #label=f'Gene {indiv.genes[gene].id}')

    plt.grid(linestyle=':')
    plt.xlabel('Iteration steps', fontsize='large')
    plt.ylabel('Gene expression level', fontsize='large')
    
    xmin, xmax = plt.xlim()
    half_expr = (1 + np.exp(-m)) / 2
    plt.hlines(half_expr, xmin, xmax,
               linestyle=':', linewidth=1.5, color='tab:pink')#, label='Activation threshold $e_{1/2}$')
    plt.xlim(xmin, xmax)
    
    #plt.legend(fontsize='large')

    if plot_title:
        plt.title(plot_title)

    plt.tight_layout()
    if plot_name:
        plt.savefig(plot_name + '.pdf', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()


In [ ]:
def make_random_indiv(nb_genes=nb_genes, default_basal_expression=None, nb_mut=0):
    genes = evotsc.Gene.generate(intergene=intergene,
                                 length=gene_length,
                                 nb_genes=nb_genes,
                                 default_basal_expression=default_basal_expression,
                                 rng=rng)
    
    indiv = evotsc.Individual(genes=genes,
                              interaction_dist=interaction_dist,
                              interaction_coef=interaction_coef,
                              sigma_basal=sigma_basal,
                              sigma_opt=sigma_opt,
                              epsilon=epsilon,
                              m=m,
                              selection_coef=selection_coef,
                              rng=rng)
    
    mutation = evotsc.Mutation(basal_sc_mutation_prob=basal_sc_mutation_prob,
                               basal_sc_mutation_var=basal_sc_mutation_var,
                               intergene_poisson_lam=intergene_poisson_lam,
                               intergene_mutation_var=intergene_mutation_var,
                               inversion_poisson_lam=inversion_poisson_lam)
    
    for i_mut in range(nb_mut):
        indiv.mutate(mutation)
    
    return indiv

# Random individual: influence of environmental supercoiling on final gene expression levels

In [ ]:
init_indiv = make_random_indiv(nb_genes=20, nb_mut=25)
# Set initial expression to e1/2 but keep the individual obtained without passing 
# default_basal_expression to make_random_indiv, for RNG purposes
for gene in init_indiv.genes:
    gene.basal_expression = default_basal_expression

In [ ]:
plot_expr(init_indiv, sigma_env=0, plot_title='', plot_name='random_gene_expr')

In [ ]:
evotsc_plot.plot_genome_and_tsc(init_indiv, sigma=0, coloring_type='by-id', print_ids=True, id_interval=1,
                                mid_gene_id=True, naming_type='alpha', show_bar=True, 
                                plot_name='random_genome_and_tsc.png')